In [80]:
def return_ner_tagged_sentence(origin_text, tokens, labels):
    ner_tagged_sentence = ''
    ner_tagged_id_list = []
    
    #divide label per character token
    #space를 제외한 모든 글자에 대한 라벨을 붙이기
    #abc : B-LOC → a : B-LOC, b : I-LOC, c : I-LOC
    char_labels = []
    char_tokens = []
    for token, label in zip(tokens, labels):
        if token == '<unk>': #이 토큰에 대한 글자수를 알 수 있나? 그럼 끝나는데
            if label.startswith("B-"):
                print("unk인데 태깅되어 있는 경우 발생")
                print(origin_text)
                print(token, label)
            # char_tokens.append('<unk>')
            # char_labels.append('O')
            continue
            
        if label == 'O':
            for i in range(len(token)):
                #한글자씩 분해시킨 토큰
                char_token = token[i]
                char_label = 'O'
                if char_token != "#":
                    char_labels.append(char_label)
                    char_tokens.append(char_token)
        elif label.startswith("B-"):
            entity = label[2:]
            for i in range(len(token)):
                char_token = token[i]
                if char_token != "#":
                    if i == 0:
                        char_label = "B-"+ entity
                    elif i > 0:
                        char_label = "I-"+ entity
                elif char_token == "#":
                    if i < 2:
                        continue
                    elif i == 2:
                        char_label = "B-"+ entity
                    elif i > 2:
                        char_label = "I-"+ entity
                        
                char_labels.append(char_label)
                char_tokens.append(char_token)
        elif label.startswith("I-"):
            entity = label[2:]
            for i in range(len(token)):
                char_token = token[i]
                char_label = "I-"+entity
                if char_token != "#":
                    char_tokens.append(char_token)
                    char_labels.append(char_label)

    #process for <unk>
    char_tokens_processed = []
    char_labels_processed = []
    c_i = 0
    for i in range(len(origin_text)):
        #space가 아닌 글자에 대해서
        origin_char = origin_text[i]
        if origin_char != ' ':
            #is_unk = True
            # try:
            if c_i == len(char_tokens):
                char_tokens_processed.append('<unk>')
                char_labels_processed.append('O')
                # print(char_tokens_processed)
                # print(char_labels_processed)
                continue

            if origin_char == char_tokens[c_i]:
                #is_unk = False #해당 origin_char는 unk이 아님
                char_tokens_processed.append(char_tokens[c_i])
                char_labels_processed.append(char_labels[c_i])
                #if c_i < len(char_tokens)-1:
                c_i += 1
                # print(char_tokens_processed)
                # print(char_labels_processed)
                

            elif origin_char != char_tokens[c_i]:
                char_tokens_processed.append('<unk>')
                char_labels_processed.append('O')
                # print(char_tokens_processed)
                # print(char_labels_processed)
            # except:
            #     print("unk error")
            #     print(origin_text)
            

    # for token, label in zip(char_tokens_processed, char_labels_processed):
    #     print(f"token {token} | label : {label}")
    char_tokens = char_tokens_processed
    char_labels = char_labels_processed
        
    #merge char tokens
    text_len = len(origin_text)
    left = 0
    right = 0
    char_id = 0 #char_tokens, char_labels 를 스캔하는 id
    
    try:
        while right < text_len: #종료조건 완벽하지 않음
            #space가 아닌 char에 대해
            if origin_text[right] == ' ':
                ner_tagged_sentence += origin_text[right]
                right += 1
            elif origin_text[right] != ' ':
                #항상 origin_text[right] 와 char_tokens[char_id] 가 동일함
                if char_labels[char_id].startswith('I-'): #잘못된 예측 I가 먼저 나온 상황
                    ner_tagged_sentence += origin_text[right]
                    char_id += 1 #개체명으로 인식하지 않고 넘기기
                    right += 1

                elif char_labels[char_id] == 'O':
                    ner_tagged_sentence += origin_text[right]
                    char_id += 1
                    right += 1

                elif char_labels[char_id].startswith('B-'): #여기서는 작업마치고 left, right 저장해야함
                    entity = char_labels[char_id][2:]
                    pred_label = 'I-'+entity
                    left = right
                    right += 1
                    char_id += 1
                    while True:
                        if origin_text[right] == ' ':
                            right += 1
                        elif origin_text[right] != ' ':
                            if char_labels[char_id] == pred_label:
                                if char_id == len(char_labels)-1:
                                    ner_tagged_sentence += (origin_text[left:right+1]+'['+entity+']')
                                    ner_tagged_id_list.append([left,right])
                                    right += 1
                                    break
                                else:
                                    char_id += 1
                                    right += 1
                            elif char_labels[char_id] != pred_label:
                                if origin_text[right-1] == ' ':
                                    ner_tagged_sentence += (origin_text[left:right-1]+'['+entity+']'+' ')
                                    ner_tagged_id_list.append([left,right-2])
                                else:
                                    ner_tagged_sentence += (origin_text[left:right]+'['+entity+']')
                                    ner_tagged_id_list.append([left,right-1])
                                left = right
                                break
    except:
        print('ERROR')
        print(origin_text)
        
    
    # for char_token, char_label in zip(char_tokens, char_labels):
    #     print(f"token : {char_token}, label : {char_label}")
    return ner_tagged_sentence, ner_tagged_id_list

In [82]:
'''
origin_text : 입력 문장
origin_tokens : tokenization 과정을 거쳤지만 <unk>인 경우에도 <unk>으로 변경 안 한 토큰
tokens : tokenization 과정을 거치고 <unk>인 경우를 <unk>으로 변경한 토큰들 (이전)
labels : origin_tokens, tokens에 대한 라벨
'''
def return_ner_tagged_sentence_plus(origin_text, origin_tokens, labels):
    ner_tagged_sentence = ''
    ner_tagged_id_list = []
    
    #divide label per character token
    #space를 제외한 모든 글자에 대한 라벨을 붙이기
    #abc : B-LOC → a : B-LOC, b : I-LOC, c : I-LOC
    char_labels = []
    char_tokens = []
    for token, label in zip(origin_tokens, labels):
        if label == 'O':
            for i in range(len(token)):
                #한글자씩 분해시킨 토큰
                char_token = token[i]
                char_label = 'O'
                if char_token != "#":
                    char_labels.append(char_label)
                    char_tokens.append(char_token)
        elif label.startswith("B-"):
            entity = label[2:]
            for i in range(len(token)):
                char_token = token[i]
                if char_token != "#":
                    if i == 0:
                        char_label = "B-"+ entity
                    elif i > 0:
                        char_label = "I-"+ entity
                elif char_token == "#":
                    if i < 2:
                        continue
                    elif i == 2:
                        char_label = "B-"+ entity
                    elif i > 2:
                        char_label = "I-"+ entity
                        
                char_labels.append(char_label)
                char_tokens.append(char_token)
        elif label.startswith("I-"):
            entity = label[2:]
            for i in range(len(token)):
                char_token = token[i]
                char_label = "I-"+entity
                if char_token != "#":
                    char_tokens.append(char_token)
                    char_labels.append(char_label)
        
    #merge char tokens
    text_len = len(origin_text)
    left = 0
    right = 0
    char_id = 0 #char_tokens, char_labels 를 스캔하는 id
    
    try:
        while right < text_len: #종료조건 
            #space가 아닌 char에 대해
            if origin_text[right] == ' ':
                ner_tagged_sentence += origin_text[right]
                right += 1
            elif origin_text[right] != ' ':
                #항상 origin_text[right] 와 char_tokens[char_id] 가 동일함
                if char_labels[char_id].startswith('I-'): #잘못된 예측 I가 먼저 나온 상황
                    ner_tagged_sentence += origin_text[right]
                    char_id += 1 #개체명으로 인식하지 않고 넘기기
                    right += 1

                elif char_labels[char_id] == 'O':
                    ner_tagged_sentence += origin_text[right]
                    char_id += 1
                    right += 1

                elif char_labels[char_id].startswith('B-'): #여기서는 작업마치고 left, right 저장해야함
                    entity = char_labels[char_id][2:]
                    pred_label = 'I-'+entity
                    left = right
                    right += 1
                    char_id += 1
                    while True:
                        if origin_text[right] == ' ':
                            right += 1
                        elif origin_text[right] != ' ':
                            if char_labels[char_id] == pred_label:
                                if char_id == len(char_labels)-1:
                                    ner_tagged_sentence += (origin_text[left:right+1]+'['+entity+']')
                                    ner_tagged_id_list.append([left,right,entity])
                                    right += 1
                                    break
                                else:
                                    char_id += 1
                                    right += 1
                            elif char_labels[char_id] != pred_label:
                                if origin_text[right-1] == ' ':
                                    ner_tagged_sentence += (origin_text[left:right-1]+'['+entity+']'+' ')
                                    ner_tagged_id_list.append([left,right-2,entity])
                                else:
                                    ner_tagged_sentence += (origin_text[left:right]+'['+entity+']')
                                    ner_tagged_id_list.append([left,right-1,entity])
                                left = right
                                break
    except:
        print('ERROR')
        print(origin_text)
        
    
    # for char_token, char_label in zip(char_tokens, char_labels):
    #     print(f"token : {char_token}, label : {char_label}")
    return ner_tagged_sentence, ner_tagged_id_list

In [64]:
origin_text = '깐쇼새우 만드는 법, 이연복 셰프 비법은 "튀김옷 반죽에 식용류"'
tokens = ['깐', '쇼', '새우', '만드', '는', '법', ',', '이연', '##복', '셰프', '비법', '은', '<unk>', '튀김', '##옷', '반죽', '에', '식용', '류', '<unk>']
labels = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
ner_tagged_sentence, ner_tagged_id_list = return_ner_tagged_sentence(origin_text, tokens,labels)
print(ner_tagged_sentence)
for item in ner_tagged_id_list:
    start, end = item
    print(origin_text[start:end+1])

깐쇼새우 만드는 법, 이연복[PER] 셰프 비법은 "튀김옷 반죽에 식용류"
이연복


In [65]:
origin_text = '신카이 마코토 감독ㆍ배우 한예리··· ‘너의 이름은’ 메가토크에서 만나요!'
tokens = ['신', '##카이', '마코토', '감독', '<unk>', '한', '##예', '##리', '·', '·', '·', '<unk>', '너', '의', '이름', '은', '<unk>', '메가', '토크', '에서', '만나', '요', '!']
labels = ['B-PER', 'I-PER', 'I-PER', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'B-POH', 'I-POH', 'I-POH', 'I-POH', 'O', 'B-POH', 'I-POH', 'O', 'O', 'O', 'O']
ner_tagged_sentence, ner_tagged_id_list = return_ner_tagged_sentence(origin_text, tokens,labels)
print(ner_tagged_sentence)
for item in ner_tagged_id_list:
    start, end = item
    print(origin_text[start:end+1])

신카이 마코토[PER] 감독ㆍ배우 한예리[PER]··· ‘너의 이름은[POH]’ 메가토크[POH]에서 만나요!
신카이 마코토
한예리
너의 이름은
메가토크


In [90]:
print(os.getcwd())
os.path.join(os.path.dir(os.getcwd())+"data","test.tsv")

/home/ubuntu/bowon_ko


AttributeError: module 'posixpath' has no attribute 'dir'

In [94]:
import csv
import os
answers = []
with open('kobort/data/test.tsv', "r", encoding="utf-8") as f:
    lines = csv.reader(f, delimiter='\t',)
    for i, line in enumerate(lines):
        if i>0:
            #print(line)
            origin_text, _, tokenized_text, label_str = line
            tokens = tokenized_text.split()
            labels = label_str.split()
            #print(origin_text)
            #print(tokens)
            #print(len(tokens))
            #print(labels)
            #print(len(labels))
            tagged_sentence, tagged_id_list = return_ner_tagged_sentence_plus(origin_text, tokens,labels)
            #print(tagged_sentence)
            answers.append([origin_text,tagged_sentence])
            #print(tagged_id_list)
            #print()

for i, answer in enumerate(answers):
    if i<2:
        print(answer[0], answer[1])

with open("kobort/data/answer2.tsv", "w", encoding="utf-8") as f:
    field_names = ["origin_text", "ner_tagged_text"]
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(field_names)
    for answer in answers:
        writer.writerow([answer[0], answer[1]])
#            {'origin_text':answer[0], 'ner_tagged_text':answer[1]})
        

ERROR
지난 14일 방송된 KBS 2TV '불후의 명곡' 왕중왕전에서는 '쇼쇼쇼, 별들의 귀환' 2편이 꾸며졌다.
ERROR
사진·글 = 임현동 기자
ERROR
우진원 디자이너는 허벅지부터 발끝까지 지퍼가 길게 달린 통 넓은 청바지와 발목까지 내려오는 길이의 통 넓은 청치마를 ‘로켓런치’ 패션쇼에 올렸다.
ERROR
유승호와 솔라의 달콤한 입맞춤을 예고한 ‘상상고양이’ 2회는 1일 오후 8시 50분에 방송된다.
ERROR
새누리 무리는 새로운 세상을 연다는 의미에서 '개(開)'자를 써서 개누리라 불리며 공주를 수호하는 데 여념하였다.
ERROR
순끼 작가의 동명 웹툰을 원자으로 한 '치즈인더트랩'은 모든 것이 완벽한 남자 유정(박해진 분)과 평범하지만 예민한 그의 대학 후배 홍설(오연서 분), 그리고 백인호(박기웅 분)등이 대학 캠퍼스에서 벌어지는 일을 그린 작품. 산다라박, 오종혁, 문지윤 등도 출연한다.
ERROR
"지하 3층에 거꾸로 파묻혔지만…" 강남 건물붕괴 인부2명 전원구조
ERROR
손연재 개인종합 사상 첫 우승… 72.55점, 4종목 모두 18점 넘어
ERROR
손흥민 골, ‘400억 가치 증명’ 3경기면 충분했다…
ERROR
대표 상품은 올 초 교보생명이 VIP 수준을 넘어 ‘VVIP’ 전용으로 내놓은 ‘교보노블리에종신보험’이다.
ERROR
마르셀 회장은 “서울시와 2년간 협의한 끝에 20년 동안 임대하게 됐다”며 “80년 가까이 된 건물 내부를 리모델링하고 최신 음향·조명 시설을 설치하는 등 총 190억원을 투자했다”고 밝혔다.
ERROR
■ 동탄2신도시 등 뉴스테이 사업자 공모국토교통부와 주택도시보증공사는 경기 화성시 동탄2신도시와 경기 수원시 호매실지구에 기업형임대주택(뉴스테이)을 공급할 사업자를 21일 공모한다고 20일 밝혔다.
ERROR
‘폭염주의보 발효 태풍 할롤라 소멸’제12호 태풍 ‘할롤라’가 27일 오전 0시를 기해 소멸한 가운데, 부산 경남과 강원 동해안에 폭염주의보가 내려졌다.
ERROR
김구라는 다시 “그럼 박원순 시장

In [67]:
origin_text = '스포츠동아 김민정 기자 ricky337@donga.com'
tokens = ['스포츠', '##동', '##아', '김민', '##정', '기자', 'r', '##ick', '##y', '33', '##7', '@', 'do', '##ng', '##a', '.', 'com']
labels = ['B-ORG', 'I-ORG', 'I-ORG', 'B-PER', 'I-PER', 'O', 'B-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH', 'I-POH']
ner_tagged_sentence, ner_tagged_id_list = return_ner_tagged_sentence(origin_text, tokens,labels)
print(ner_tagged_sentence)
for item in ner_tagged_id_list:
    start, end = item
    print(origin_text[start:end+1])

token : 스, label : B-ORG
token : 포, label : I-ORG
token : 츠, label : I-ORG
token : 동, label : I-ORG
token : 아, label : I-ORG
token : 김, label : B-PER
token : 민, label : I-PER
token : 정, label : I-PER
token : 기, label : O
token : 자, label : O
token : r, label : B-POH
token : i, label : I-POH
token : c, label : I-POH
token : k, label : I-POH
token : y, label : I-POH
token : 3, label : I-POH
token : 3, label : I-POH
token : 7, label : I-POH
token : @, label : I-POH
token : d, label : I-POH
token : o, label : I-POH
token : n, label : I-POH
token : g, label : I-POH
token : a, label : I-POH
token : ., label : I-POH
token : c, label : I-POH
token : o, label : I-POH
token : m, label : I-POH
스포츠동아[ORG] 김민정[PER] 기자 ricky337@donga.com[POH]
스포츠동아
김민정
ricky337@donga.com


In [23]:
origin_text = '오늘 고보원은 9시에 티맥스로 출근했다'
tokens = ['오늘', '고', '##보', '##원', '은', '9', '시', '에', '티', '##맥스', '로', '출근', '했', '다']
labels = ['O', 'B-PER', 'I-PER', 'I-PER', 'O', 'B-TIM', 'I-TIM', 'O', 'B-ORG', 'I-ORG', 'O', 'O', 'O', 'O']
ner_tagged_sentence, ner_tagged_id_list = return_ner_tagged_sentence(origin_text, tokens,labels)
print(ner_tagged_sentence)
for item in ner_tagged_id_list:
    start, end = item
    print(origin_text[start:end+1])

token : 오, label : O
token : 늘, label : O
token : 고, label : B-PER
token : 보, label : I-PER
token : 원, label : I-PER
token : 은, label : O
token : 9, label : B-TIM
token : 시, label : I-TIM
token : 에, label : O
token : 티, label : B-ORG
token : 맥, label : I-ORG
token : 스, label : I-ORG
token : 로, label : O
token : 출, label : O
token : 근, label : O
token : 했, label : O
token : 다, label : O
오늘 고보원[PER]은 9시[TIM]에 티맥스[ORG]로 출근했다
고보원
9시
티맥스


In [22]:
origin_text = "세련미 가미한 공격 우리은행 박혜진, 임영희, 양지희는 공격에도 능하다."
tokens = ['세련', '##미', '가', '미', '한', '공격', '우리', '##은행', '박', '##혜진', ',', '임영', '##희', ',', '양지', '##희', '는', '공격', '에', '도', '능하', '다', '.']
labels = ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
return_ner_tagged_sentence(origin_text, tokens,labels)

token : 세, label : O
token : 련, label : O
token : 미, label : O
token : 가, label : O
token : 미, label : O
token : 한, label : O
token : 공, label : O
token : 격, label : O
token : 우, label : B-ORG
token : 리, label : I-ORG
token : 은, label : I-ORG
token : 행, label : I-ORG
token : 박, label : B-PER
token : 혜, label : I-PER
token : 진, label : I-PER
token : ,, label : O
token : 임, label : B-PER
token : 영, label : I-PER
token : 희, label : I-PER
token : ,, label : O
token : 양, label : B-PER
token : 지, label : I-PER
token : 희, label : I-PER
token : 는, label : O
token : 공, label : O
token : 격, label : O
token : 에, label : O
token : 도, label : O
token : 능, label : O
token : 하, label : O
token : 다, label : O
token : ., label : O


('세련미 가미한 공격 우리은행[ORG] 박혜진[PER], 임영희[PER], 양지희[PER]는 공격에도 능하다.',
 [[11, 14], [16, 18], [21, 23], [26, 28]])

In [76]:
import csv
from tokenizer import make_tokens
answers = []
with open("test.tsv", "r", encoding="utf-8") as f:
    lines = csv.reader(f, delimiter='\t',)
    for i, line in enumerate(lines):
        if i>0:
            #print(line)
            origin_text, _, _, label_str = line
            morphed_text = make_tokens(origin_text, model_name="mecab")
            tokenized_text = make_tokens(origin_text, model_name="wp-mecab")
            tokenized_text = ' '.join(tokenized_text)
            answers.append([origin_text, morphed_text, tokenized_text, label_str])

for i, answer in enumerate(answers):
    if i<2:
        print(answer[0], answer[1], answer[2], answer[3])

with open("test_v1.1.tsv", "w", encoding="utf-8") as f:
    field_names = ["original_text", "morphed_text", "tokenize", "bio_tagging"]
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(field_names)
    for answer in answers:
        writer.writerow([answer[0], answer[1], answer[2], answer[3]])

        

AttributeError: module 'tweepy' has no attribute 'StreamListener'

In [75]:
!pip install tweepy==3.10.0

  Using cached tweepy-3.10.0-py2.py3-none-any.whl (30 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Attempting uninstall: tweepy
    Found existing installation: tweepy 4.4.0
    Uninstalling tweepy-4.4.0:
      Successfully uninstalled tweepy-4.4.0
